In [1]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('Resources/train10000.csv')
#chunksize returns as a loop through the dataset. 
data.head()
#data[0:1000].to_csv("Resources/train1000.csv", index=False)
#data[0:10000].to_csv("Resources/train10000.csv", index=False)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [3]:
X = data[["MachineIdentifier", "ProductName", "EngineVersion", "AppVersion"]]
y = data["HasDetections"].values.reshape(-1, 1)
print(X.shape, y.shape)

(10000, 4) (10000, 1)


In [7]:
data_temp = X.copy()

data_binary_encoded = pd.get_dummies(data_temp)
data_binary_encoded.head()

,MachineIdentifier_0000028988387b115f69f31a3bf04f09,MachineIdentifier_000007535c3f730efa9ea0b7ef1bd645,MachineIdentifier_000007905a28d863f6d0d597892cd692,MachineIdentifier_00000b11598a75ea8ba1beea8459149f,MachineIdentifier_000014a5f00daa18e76b81417eeb99fc,MachineIdentifier_000016191b897145d069102325cab760,MachineIdentifier_0000161e8abf8d8b89c5ab8787fd712b,MachineIdentifier_000019515bc8f95851aff6de873405e8,MachineIdentifier_00001a027a0ab970c408182df8484fce,MachineIdentifier_00001a18d69bb60bda9779408dcf02ac,...,AppVersion_4.9.10586.1106,AppVersion_4.9.10586.494,AppVersion_4.9.10586.589,AppVersion_4.9.10586.672,AppVersion_4.9.10586.839,AppVersion_4.9.10586.873,AppVersion_4.9.10586.916,AppVersion_4.9.10586.962,AppVersion_4.9.10586.965,AppVersion_4.9.218.0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.feature_selection import RFECV #reduce number of features into the model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, CategoricalEncoder

In [5]:
class PandasDummies(BaseEstimator, TransformerMixin):
    def transform(self, X, *_):
        return pd.get_dummies(X)
    
    def fit(self, *_):
        return self

X = data[["MachineIdentifier", "ProductName", "EngineVersion", "AppVersion"]]
y = data["HasDetections"]
X = pd.get_dummies(X,drop_first=True)
#.values.reshape(-1, 1)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

model = make_pipeline(PandasDummies(), StandardScaler())

(10000, 10092)
(10000,)


In [6]:
lr = LogisticRegressionCV(n_jobs=5,random_state=42,class_weight='balanced')

In [7]:
lr = RFECV(lr,step=5,min_features_to_select=10,scoring='accuracy',n_jobs=3)

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr.predict(X_test)

In [ ]:
lr.

In [ ]:
lr.score(X_test)

In [ ]:
predictions = lr.predict(X_test)
r2 = lr.score(X_test, y_test)

print(f"MSE: {MSE}, R2: {r2}")